In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

In [2]:
df = pds.random_data(15_000)
df.head()

row_num,feature_1,feature_2,feature_3
u64,f64,f64,f64
0,0.119315,0.431571,0.784612
1,0.811068,0.711681,0.681357
2,0.223527,0.996492,0.043001
3,0.923635,0.796295,0.573309
4,0.299232,0.20504,0.732848


In [ ]:
print(
    
df.select(
    pl.col("row_num"),
    pds.query_radius_ptwise(
        pl.col("feature_1"), pl.col("feature_2"), pl.col("feature_3"), # Columns used as the coordinates in n-d space
        index = pl.col("row_num"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head()

)

In [ ]:
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True

In [ ]:
import scipy

In [ ]:
df.select(pl.col("feature_1").num.rfft())

In [ ]:
df.select(pl.col("feature_1").num.rfft2())